In [1]:
import json
import transformers
import torch
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, TfidfRetriever, FilterRetriever, DensePassageRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers
from docx import Document

In [2]:
import import_ipynb
from Load_and_Print import read_qas, print_result

importing Jupyter notebook from Load_and_Print.ipynb


In [3]:
def load_test_data(file):
    with open(file) as jfile:
        data = json.load(jfile) 
        samples = []
    for i in range (0,10):
        contract = "".join(data["data"][i]["paragraphs"][0]["context"].split("\n"))
        samples.append(contract)
    return samples

In [4]:
def read_qas(file):
    qas_doc = Document(file)
    questions = []
    for para in qas_doc.paragraphs:
        question = {}
        k, v = para.text.split(":", 1)
        question[k.strip()] = v.strip()
        questions.append(question)
    return questions

In [5]:
def test_model1(qs, c):
    model = transformers.AutoModelForQuestionAnswering.from_pretrained("akdeniz27/roberta-large-cuad")
    tokenizer = transformers.AutoTokenizer.from_pretrained("akdeniz27/roberta-large-cuad")
    qa = transformers.pipeline("question-answering", model=model, tokenizer=tokenizer)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        querry = list(qs[j].values())[0]
        res = qa({
        "question": querry,
        "context": c
        })
        print(res)
        ans.append(res)
    return ans

In [6]:
def test_model2(qs, c):
    model = transformers.AutoModelForQuestionAnswering.from_pretrained("Rakib/roberta-base-on-cuad")
    tokenizer = transformers.AutoTokenizer.from_pretrained("Rakib/roberta-base-on-cuad")
    qa = transformers.pipeline('question-answering', model=model, tokenizer=tokenizer)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        querry = list(qs[j].values())[0]
        res = qa({
        "question": querry,
        "context": c
        })
        print(res)
        ans.append(res)
    return ans

In [26]:
def test_model3(qs, c, doc_store):
    retriever = BM25Retriever(doc_store) 
    reader = FARMReader(model_name_or_path='Rakib/roberta-base-on-cuad',
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return ans

In [8]:
def test_model4(qs, c, doc_store):
    retriever = BM25Retriever(doc_store)
    reader = FARMReader(model_name_or_path='akdeniz27/roberta-large-cuad',
                        
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return ans

In [9]:
def test_model5(qs, c, doc_store):
    retriever = TfidfRetriever(doc_store) 
    reader = FARMReader(model_name_or_path='Rakib/roberta-base-on-cuad',
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return ans

In [10]:
def test_model6(qs, c, doc_store):
    retriever = FilterRetriever(doc_store) 
    reader = FARMReader(model_name_or_path='Rakib/roberta-base-on-cuad',
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return ans

In [11]:
def test_model7(qs, c, doc_store):
    retriever = BM25Retriever(doc_store) 
    reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return ans

In [12]:
def test_model8(qs, c, doc_store):
    retriever = DensePassageRetriever(
        document_store=doc_store,
        query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
        passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
        use_gpu=True, embed_title=True)
    doc_store.update_embeddings(retriever=retriever)
    retriever = BM25Retriever(doc_store) 
    reader = FARMReader(model_name_or_path='Rakib/roberta-base-on-cuad',
                        context_window_size=1500,
                        use_gpu=True)
    qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)
    ans = []
    for j in range(0, len(qs)):
        print (f"qs{j+1}")
        query = list(qs[j].values())[0]
        res = qa.run(
            query=query, params={"Reader": {"top_k": 1},"Retriever": {"top_k": 1}}
        )
        print_answers(res, details = "minimum")
        ans.append(res)
    return res

In [13]:
def set_doc_store(c, i):
    doc_store = ElasticsearchDocumentStore(
        host='localhost',
        username='elastic', password='hrAezzG2su2WPqvOu8J9',
        index= str(i)
    )
    data_json = [
    {
        'content':c,
        'meta':{
            'source': f"contract {str(i)}"
            }
        }
    ]
    doc_store.write_documents(data_json, index = str(i))
    return doc_store

In [14]:
qas = read_qas("./Questions.docx")
samples = load_test_data("./test.json")

In [15]:
def test1():
    print("Model1 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        test_model1(qas, context)

In [16]:
def test2():
    print("Model2 Result")
    for i in range(0,5):
        context = samples[i]
        print(f"contract{i+1}")
        test_model2(qas, context)

In [17]:
def test3():
    print("Model3 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+20)
        test_model3(qas, context, doc_store)
        doc_store.delete_index(str(i+20))

In [18]:
def test4():
    print("Model4 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+30)
        test_model4(qas, context, doc_store)
        doc_store.delete_index(str(i+30))

In [19]:
 def test5():
    print("Model5 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+40)
        test_model5(qas, context, doc_store)
        doc_store.delete_index(str(i+40))

In [20]:
def test6():    
    print("Model6 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+50)
        test_model6(qas, context, doc_store)
        doc_store.delete_index(str(i+50))

In [21]:
def test7():
    print("Model7 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+60)
        test_model7(qas, context, doc_store)
        doc_store.delete_index(str(i+60))

In [33]:
def test8():
    print("Model8 Result")
    for i in range(0, 5):
        context = samples[i]
        print(f"contract{i+1}")
        doc_store = set_doc_store(context, i+70)
        test_model8(qas, context, doc_store)
        doc_store.delete_index(str(i+70))

In [23]:
if __name__ == '__main__':
    test1()

Model1 Result
contract1
qs1
{'score': 0.7985469698905945, 'start': 14, 'end': 29, 'answer': 'SUPPLY CONTRACT'}
qs2
{'score': 0.32828816771507263, 'start': 69, 'end': 117, 'answer': 'Shenzhen LOHAS Supply Chain Management Co., Ltd.'}
qs3
{'score': 0.0012087380746379495, 'start': 69, 'end': 117, 'answer': 'Shenzhen LOHAS Supply Chain Management Co., Ltd.'}
qs4
{'score': 0.8674935698509216, 'start': 11279, 'end': 11339, 'answer': 'The Contract becomes effective after signed by both parties.'}
qs5
{'score': 0.5755215287208557, 'start': 10935, 'end': 10999, 'answer': 'The Contract is valid for 5 years, beginning from and ended on .'}
qs6
{'score': 0.07491620630025864, 'start': 10935, 'end': 10997, 'answer': 'The Contract is valid for 5 years, beginning from and ended on'}
qs7
{'score': 8.360096614978829e-08, 'start': 10935, 'end': 10997, 'answer': 'The Contract is valid for 5 years, beginning from and ended on'}
qs8
{'score': 0.00010890018893405795, 'start': 10808, 'end': 10809, 'answer': '

In [24]:
if __name__ == '__main__':
    test2()

Model2 Result
contract1
qs1
{'score': 0.3929581940174103, 'start': 14, 'end': 29, 'answer': 'SUPPLY CONTRACT'}
qs2
{'score': 0.04707128554582596, 'start': 1037, 'end': 1064, 'answer': 'YICHANGTAI" or "LEHEYUAN"),'}
qs3
{'score': 0.09759918600320816, 'start': 10935, 'end': 10999, 'answer': 'The Contract is valid for 5 years, beginning from and ended on .'}
qs4
{'score': 0.3961285948753357, 'start': 11279, 'end': 11339, 'answer': 'The Contract becomes effective after signed by both parties.'}
qs5
{'score': 0.9683458805084229, 'start': 10935, 'end': 10999, 'answer': 'The Contract is valid for 5 years, beginning from and ended on .'}
qs6
{'score': 0.0008004933479242027, 'start': 831, 'end': 874, 'answer': 'During the validity term of this agreement,'}
qs7
{'score': 0.0323307067155838, 'start': 10935, 'end': 10999, 'answer': 'The Contract is valid for 5 years, beginning from and ended on .'}
qs8
{'score': 0.0057416339404881, 'start': 10641, 'end': 10707, 'answer': "It will be governed by th

In [27]:
if __name__ == '__main__':
    test3()

Model3 Result
contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.74s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'SUPPLY CONTRACT',
        'context': 'Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The '
                   'buyer/End-User: Shenzhen LOHAS Supply Chain Management '
                   'Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The '
                   'Contract is concluded and signed by the Buyer and Seller '
                   'on , in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will b

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.83s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD',
        'context': 'act is concluded and signed by the Buyer and Seller on , '
                   'in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will be subject to the provisions of agreement. '
                   'Any modification, supplementary, give up should been '
                   'written records, only to be valid by buyer

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY.

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.03s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY. This provision applies '
                   'o

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.80s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'That includes contracts of insurance, separate accounts, '
                  'and cash and investment securities.',
        'context': 'ENDORSEMENTContract Number: ENDORSEMENTEffective Date: '
                   'July 1, 2019Thrivent Life Insurance Company has been '
                   'dissolved. (Thrivent Life Insurance Company was formerly '
                   'known as Lutheran Brotherhood Variable Insurance Products '
                   'Company.) All assets and related liabilities of Thrivent '
                   'Life Insurance Company have been transferred to Thrivent '
                   'Financial for Lutherans. That includes contracts of '
                   'insurance, separate accounts, and cash and investment '
   

In [28]:
if __name__ == '__main__':
    test4()

Model4 Result
contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:19<00:00, 19.05s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'SUPPLY CONTRACT',
        'context': 'Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The '
                   'buyer/End-User: Shenzhen LOHAS Supply Chain Management '
                   'Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The '
                   'Contract is concluded and signed by the Buyer and Seller '
                   'on , in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will b

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.67s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'Shenzhen LOHAS Supply Chain Management Co., Ltd.',
        'context': 'Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The '
                   'buyer/End-User: Shenzhen LOHAS Supply Chain Management '
                   'Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The '
                   'Contract is concluded and signed by the Buyer and Seller '
                   'on , in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not st

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'The person(s) named as the Insured(s) or the Annuitant(s) '
                  'is a benefit member of the Society. Rights and privileges '
                  'of membership are set forth in the Articles of '
                  'Incorporation and Bylaws of the Society. These rights and '
                  'privileges are separate from the ownership of this '
                  'contract.',
        'context': 'ts provided through the General Account. If the solvency '
                   'of the Society becomes impaired, you may be required to '
                   'make an extra payment. The Board of Directors will '
                   'determine the amou

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.57s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'If',
        'context': 'his contract remain unchanged except as described '
                   'below.You may direct inquiries to:Thrivent Financial for '
                   'Lutherans 4321 N. Ballard Road Appleton, WI 54919-0001 '
                   '(800) 847-4836The following amendments apply in lieu of '
                   'any contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY. This provision applies '
                   'only to benefits provided through the General Account. If '
                   'the solvency of the Society becomes impaired, you may be '
                 

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.97s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'Thrivent Life Insurance Company',
        'context': 'read:We, we, Our, our, Us, us, Society. Thrivent Financial '
                   'for Lutherans.ENTIRE CONTRACTIn the ENTIRE CONTRACT '
                   'section, the first phrase and numbered list are amended to '
                   'read:The Entire Contract consists of: 1) This contract '
                   'including any attached riders, amendments, or '
                   'endorsements; 2) The Application attached to this '
                   'contract; and 3) The Articles of Incorporation and Bylaws '
                   'of the Society and all amendments to them. Benefits will '
                   'not be reduced or eliminated by any future amendments to '
             

In [29]:
if __name__ == '__main__':   
    test5()

Model5 Result
contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.54s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'SUPPLY CONTRACT',
        'context': 'Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The '
                   'buyer/End-User: Shenzhen LOHAS Supply Chain Management '
                   'Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The '
                   'Contract is concluded and signed by the Buyer and Seller '
                   'on , in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will b

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.43s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD',
        'context': 'act is concluded and signed by the Buyer and Seller on , '
                   'in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will be subject to the provisions of agreement. '
                   'Any modification, supplementary, give up should been '
                   'written records, only to be valid by buyer

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY.

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.30s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY. This provision applies '
                   'o

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.45s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'That includes contracts of insurance, separate accounts, '
                  'and cash and investment securities.',
        'context': 'ENDORSEMENTContract Number: ENDORSEMENTEffective Date: '
                   'July 1, 2019Thrivent Life Insurance Company has been '
                   'dissolved. (Thrivent Life Insurance Company was formerly '
                   'known as Lutheran Brotherhood Variable Insurance Products '
                   'Company.) All assets and related liabilities of Thrivent '
                   'Life Insurance Company have been transferred to Thrivent '
                   'Financial for Lutherans. That includes contracts of '
                   'insurance, separate accounts, and cash and investment '
   

In [30]:
if __name__ == '__main__':
    test6()

Model6 Result
contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.57s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'SUPPLY CONTRACT',
        'context': 'Exhibit 10.16 SUPPLY CONTRACT Contract No: Date: The '
                   'buyer/End-User: Shenzhen LOHAS Supply Chain Management '
                   'Co., Ltd. ADD: Tel No. : Fax No. : The seller: ADD: The '
                   'Contract is concluded and signed by the Buyer and Seller '
                   'on , in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will b

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.64s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'SHENZHEN YICHANGTAI IMPORT AND EXPORT TRADE CO., LTD',
        'context': 'act is concluded and signed by the Buyer and Seller on , '
                   'in Hong Kong. 1. General provisions 1.1 This is a '
                   'framework agreement, the terms and conditions are applied '
                   'to all purchase orders which signed by this agreement '
                   '(hereinafter referred to as the "order"). 1.2 If the '
                   'provisions of the agreement are inconsistent with the '
                   'order, the order shall prevail. Not stated in order '
                   'content will be subject to the provisions of agreement. '
                   'Any modification, supplementary, give up should been '
                   'written records, only to be valid by buyer

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY.

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.47s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY. This provision applies '
                   'o

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.60s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'That includes contracts of insurance, separate accounts, '
                  'and cash and investment securities.',
        'context': 'ENDORSEMENTContract Number: ENDORSEMENTEffective Date: '
                   'July 1, 2019Thrivent Life Insurance Company has been '
                   'dissolved. (Thrivent Life Insurance Company was formerly '
                   'known as Lutheran Brotherhood Variable Insurance Products '
                   'Company.) All assets and related liabilities of Thrivent '
                   'Life Insurance Company have been transferred to Thrivent '
                   'Financial for Lutherans. That includes contracts of '
                   'insurance, separate accounts, and cash and investment '
   

In [31]:
if __name__ == '__main__':
    test7()

Model7 Result
contract1


Downloading:   0%|          | 0.00/508 [00:00<?, ?B/s]

C:\Users\Haiying\anaconda3\envs\ml\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Haiying\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/152 [00:00<?, ?B/s]

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'This provision applies only to benefits provided through '
                  'the General Account',
        'context': ', separate accounts, and cash and investment securities. '
                   'Thrivent Financial for Lutherans now has all obligations '
                   'under this contract. It is the same as if Thrivent '
                   'Financial for Lutherans had originally issued the '
                   'contract. All references to "Thrivent Life Insurance '
                   'Company" are amended to read "Thrivent Financial for '
                   'Lutherans." All benefits and other terms of this contract '
                   'rema

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.28s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': '7/1/2019',
        'context': 'separate from the ownership of this contract.DIVIDENDS. '
                   'Each year, we will determine our divisible surplus. This '
                   "contract's share, if any, will be credited as a dividend. "
                   'Since we do not expect this contract to contribute to '
                   'divisible surplus, it is not expected that any dividends '
                   'will be credited.DIVIDEND OPTIONS. If dividends are '
                   'credited after premiums can no longer be paid under this '
                   'contract, dividends will be paid in cash. Otherwise, '
                   'dividends will be applied under the Payment of Premiu

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.44s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'All benefits and other terms of this contract remain '
                  'unchanged',
        'context': 'ontract Number: ENDORSEMENTEffective Date: July 1, '
                   '2019Thrivent Life Insurance Company has been dissolved. '
                   '(Thrivent Life Insurance Company was formerly known as '
                   'Lutheran Brotherhood Variable Insurance Products Company.) '
                   'All assets and related liabilities of Thrivent Life '
                   'Insurance Company have been transferred to Thrivent '
                   'Financial for Lutherans. That includes contracts of '
                   'insurance, separate accounts, and cash and investment '
                   'securities. Thrivent Fina

In [34]:
if __name__ == '__main__':
    test8()

Model8 Result
contract1


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating embeddings:   0%|          | 0/1 [00:00<?, ? Docs/s]

qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:04<00:00,  4.79s/ Batches]


...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY.

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.42s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'If you do not make the extra payment within 60 days from '
                  'the date we notify you of your share of the deficiency, the '
                  'amount will be charged as an indebtedness against the '
                  'contract with interest compounded at the rate of 5% per '
                  'year.',
        'context': 's 4321 N. Ballard Road Appleton, WI 54919-0001 (800) '
                   '847-4836The following amendments apply in lieu of any '
                   'contract provisions to the contrary.The following '
                   'provisions are included as part of this '
                   'contract:MAINTENANCE OF SOLVENCY. This provision applies '
                   'o

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.51s/ Batches]


Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'That includes contracts of insurance, separate accounts, '
                  'and cash and investment securities.',
        'context': 'ENDORSEMENTContract Number: ENDORSEMENTEffective Date: '
                   'July 1, 2019Thrivent Life Insurance Company has been '
                   'dissolved. (Thrivent Life Insurance Company was formerly '
                   'known as Lutheran Brotherhood Variable Insurance Products '
                   'Company.) All assets and related liabilities of Thrivent '
                   'Life Insurance Company have been transferred to Thrivent '
                   'Financial for Lutherans. That includes contracts of '
                   'insurance, separate accounts, and cash and investment '
   